# Autoencoder

- The molecules for this notebook are a subset of the [QM09 dataset](https://www.nature.com/articles/sdata201422) by R. Ramakrishnan1, P. O. Dral, M. Rupp, and O. A. von Lilienfeld *Sci. Data*, **1**, 140022 (2014).

- A total of 17,834 small molecules with 18 atoms each were selected.

<img src="https://www.mdpi.com/sensors/sensors-21-04294/article_deploy/html/images/sensors-21-04294-g003.png" alt="Illustration of an Autoencoder" aling="right" style="width: 500px;float: right;"/>

## Overview

In this notebook we will learn how to implement an autoencoder. This algorithm is a type of artificial neural network used to learn efficient codings of input data. The code is a lower-dimensional representation of the input data, typically used for dimensionality reduction or feature learning. The autoencoder learns to compress the data from the input layer into a short code, and then uncompress that code into something that closely matches the original data. This forces the autoencoder to learn the most important features of the data, and is a useful way to extract useful features from the data.

# Libraries

In [ ]:
import torch

import matplotlib.pyplot as plt

from pathlib          import Path
from torch.utils.data import DataLoader
#
### Import local libraries
#
from model import AutoEncoder

## 1. Prepare data

Fisrt we need to load our dataset. We will use the `data_2d.pth` that includes the adjacency matrix for 17,834 small molecules. Our training data will be the adjacency matrix and the target will be the same matrix. For that, we will shuffle and dive our set into 80 % for training and the remaining 20 % for validation.

In [ ]:
#
### Define data files
#
imhere  = Path.cwd()

csvfile = imhere/'data.csv'
nnfile  = imhere/'network.pth'
#
### Load dataset
#
datapth = torch.load(imhere/'data_2d.pth')
#
### Shuffle data
#
idx     = torch.randperm(datapth.shape[0])

datapth = datapth[idx]
#
### Divide into 80% training and 20% testing
#
limit   = 20*len(idx)//100

print(f'train = {len(datapth[:-limit])}, '
      f'test = {len(datapth[-limit:])}, '
      f'test ratio = {len(datapth[-limit:])/len(datapth):.2f}')

## 2. Settings and hyperparameters

Our optimization algorithm is the ADAptive Moment estimation, [Adam](https://arxiv.org/pdf/1412.6980.pdf), that is based on stochastic gradient descent. We will need to define the **learning rate** and the **weight decay**. The learning rate is a hyperparameter that controls how much we are adjusting the weights of our network with respect to the loss gradient, whereas the weight decay is a regularization term that penalizes large weights

The number of **epochs** is the number of times the learning algorithm will work through the entire training dataset. One epoch means that each sample in the training dataset has had an opportunity to update the internal model parameters.

The **batch size** is a hyperparameter that defines the number of samples to work through before updating the internal model parameters. The batch size is a compromise between the speed and the quality of the learning. The larger the batch size, the faster the learning, but the less accurate the updates. The smaller the batch size, the slower the learning, but the more accurate the updates.

Because the adjancency matrix contains only 0 and 1, we will use the Binary Cross Entropy **loss function**.
$$
\text{BCE} = -\frac{1}{N} \sum_{i=1}^N \left[ y_i \log(\hat{y}_i) + (1-y_i) \log(1-\hat{y}_i) \right]
$$
where $N$ is the number of elements in the input tensor, $y_i$ is the target tensor, and $\hat{y}_i$ is the input tensor. The loss function takes the input tensor and the target tensor as arguments and returns the loss. The loss is the average of the binary cross entropy loss of all elements in the input tensor.

The training and testing data may be used in a loop function,

~~~
for batch in training: print(batch.shape)
~~~

each loop will automatically pass a total of `batch_size` samples to the neural network.

In [ ]:
#
### Training parameters
#
learnig_rate = 1e-1
weight_decay = 0.0

epochs       = 50
batch_size   = 492 # 1, 2, 3, 4, 6, 12, 29, 41, 58, 82, 87, 116, 123, 164, 174, 246, 348, 492, 1189, 2378, 3567, 4756, 7134, 14268
test_epoch   = 10
#
### Define neural network
#
network = AutoEncoder()
#
### Optimizer and Loss
#
optimizer = torch.optim.Adam(params=network.parameters(), lr=learnig_rate, weight_decay=weight_decay)
criterion = torch.nn.BCELoss(reduction='mean')
#
### Training and testing data
#
training = DataLoader(datapth[:-limit], shuffle=True,  batch_size=batch_size)
testing  = DataLoader(datapth[-limit:], shuffle=False, batch_size=batch_size)

## 3. Training

We can now train our neural network for the total `epochs` we selected and testing it every `test_epoch` epochs.

Passing training data to the networks consists in five steps:

1. Set the gradients to zero, `optimizer.zero_grad()`.

2. Pass batch to the network, `output = network(batch)`.

3. Compute the loss, `loss = criterion(output, y)`.

4. Perform backward pass, `loss.backward()`.

5. Perform the optimization step, `optimizer.step()`.

Keep in mind that during testing you **DO NOT** want to update the gradients in your neural network. Otherwise you will leak testing information and your model will also learn from the testing set. To prevent this from happening, you need to use the `torch.no_grad()` context manager.  This will prevent the gradient from being updated. 

In [ ]:
for epoch in range(epochs):

    # your code here for the training set

    print(f'{epoch+1},train,{loss:.4f}')

    if (epoch+1)%test_epoch == 0:
        # your code here for the testing set

        print(f'{epoch+1},test,{loss:.4f}')
#
### Save model
#
torch.save(network.state_dict(), nnfile)

## 5. Test the model

Choose a random sample from the test set to compare the original and reconstructed adjacency matrices.